In [1]:
import tensorflow as tf
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, Flatten, Dropout

In [2]:
batch_size=16
epochs=10
input_shape = (21,3)
classes = 6

In [3]:
x,y=[],[]
with h5py.File('../gestures.hdf5','a') as f:
    print(f.filename)
    for i,gesture in enumerate(list(f['/'])):
        print(i,gesture)
        data = list(f['/'+gesture+'/ds1'])
        x.extend(data)
        y.extend([i]*len(data))

x=np.array(x)
y=np.array(y) 
y=tf.keras.utils.to_categorical(y,classes)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle=True)
print(x_train.shape,x_test.shape)

../gestures.hdf5
0 five_finger
1 four_finger
2 ok
3 one_finger
4 three_finger
5 two_finger
(69919, 21, 3) (7769, 21, 3)


In [11]:
model = tf.keras.models.Sequential([
    Flatten(input_shape=(21,3)),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(classes, activation='softmax')
])
model.build()
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 63)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               16384     
_________________________________________________________________
dense_14 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 6)                 1542      
Total params: 83,718
Trainable params: 83,718
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_split=0.1)
model.evaluate(x_test,y_test)

Epoch 1/10
3933/3933 [==============================] - 15s 4ms/step - loss: 0.5535 - accuracy: 0.8093 - val_loss: 0.2150 - val_accuracy: 0.9314
Epoch 2/10
3933/3933 [==============================] - 15s 4ms/step - loss: 0.2285 - accuracy: 0.9268 - val_loss: 0.1815 - val_accuracy: 0.9424
Epoch 3/10
3933/3933 [==============================] - 15s 4ms/step - loss: 0.1837 - accuracy: 0.9392 - val_loss: 0.1576 - val_accuracy: 0.9489
Epoch 4/10
3933/3933 [==============================] - 15s 4ms/step - loss: 0.1590 - accuracy: 0.9469 - val_loss: 0.1492 - val_accuracy: 0.9511
Epoch 5/10
3933/3933 [==============================] - 16s 4ms/step - loss: 0.1526 - accuracy: 0.9491 - val_loss: 0.1404 - val_accuracy: 0.9534
Epoch 6/10
3933/3933 [==============================] - 16s 4ms/step - loss: 0.1363 - accuracy: 0.9548 - val_loss: 0.1353 - val_accuracy: 0.9557
Epoch 7/10
3933/3933 [==============================] - 15s 4ms/step - loss: 0.1254 - accuracy: 0.9578 - val_loss: 0.1325 - val_ac

[0.11751742660999298, 0.9620285630226135]

In [13]:
model.save('test_model_1')

INFO:tensorflow:Assets written to: test_model_1\assets
